In [44]:
!python -c "import sys; print(sys.executable)"

/home/saadsameerkhan/anaconda3/bin/python


# Setting up mlflow server at port 1593

In [45]:
%%bash --bg

mlflow server --host 0.0.0.0 \
    --port 1593 \
    --backend-store-uri sqlite:///mlflow.db \
    --default-artifact-root ./mlruns

In [46]:
%cat MLproject

name: basic_mlflow

conda_env: conda.yaml

entry_points:
  main:
    # parameters is a key-value collection.
    parameters:
      solver_name:
        type: str
        default: "lbfgs"
      logreg_type:
        type: str
        default: "multinomial"
    command: "python new_train.py {logreg_type} {solver_name}"

# Running MLproject file 

In [47]:
%%bash
source mlflow_env_vars.sh
mlflow run .

2023/05/06 20:05:41 INFO mlflow.utils.conda: Conda environment mlflow-dd0fbdd40ba98798131458f29496394bd1a3fb33 already exists.
2023/05/06 20:05:41 INFO mlflow.projects.utils: === Created directory /tmp/tmpdnadj8kn for downloading remote URIs passed to arguments of type 'path' ===
2023/05/06 20:05:41 INFO mlflow.projects.backend.local: === Running command 'source /home/saadsameerkhan/anaconda3/bin/../etc/profile.d/conda.sh && conda activate mlflow-dd0fbdd40ba98798131458f29496394bd1a3fb33 1>&2 && python new_train.py multinomial lbfgs' in run with ID 'a0d43e51bfbf4d3e87821c1c2fcbce7e' === 
/home/saadsameerkhan/anaconda3/envs/mlflow-dd0fbdd40ba98798131458f29496394bd1a3fb33/lib/python3.11/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
Registered model 'sklearn_logreg' already exists. Creating a new version of this model...
2023/05/06 20:05:44 INFO mlflow.tracking._model_registry.client: Wait

# Checking MLmodel file of last model run
### It gives info about the model, the input output data as well as some metadata

In [48]:
%%bash
last_model_path=$(ls -tr mlruns/0/ | tail -1)
cat mlruns/0/$last_model_path/artifacts/logreg/MLmodel

artifact_path: logreg
flavors:
  python_function:
    env:
      conda: conda.yaml
      virtualenv: python_env.yaml
    loader_module: mlflow.sklearn
    model_path: model.pkl
    predict_fn: predict
    python_version: 3.11.3
  sklearn:
    code: null
    pickled_model: model.pkl
    serialization_format: cloudpickle
    sklearn_version: 1.2.2
mlflow_version: 2.3.1
model_uuid: 5d2f9556dcea4f46aa6caf6365acbfa3
run_id: a0d43e51bfbf4d3e87821c1c2fcbce7e
utc_time_created: '2023-05-06 15:05:43.389914'


# Serving the model that is in production
## so that we can give it our data
### and it give us its predictions

In [49]:
%%bash --bg
source mlflow_env_vars.sh
mlflow --version
mlflow models serve -m models:/sklearn_logreg/Production -p 1594 --env-manager=conda

# Selecting the last 2 rows of the dataset
### These rows were unseen by the model during training

In [72]:
from sklearn import datasets
wine = datasets.load_wine()
X = pd.DataFrame(wine.data, columns = wine.feature_names)   # dataframe with all feature columns
y = pd.DataFrame(wine.target, columns = ['encoded_class'])   # dataframe with target column
X['target'] = y
test_df = X.iloc[-2:]
test_df

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline,target
176,13.17,2.59,2.37,20.0,120.0,1.65,0.68,0.53,1.46,9.3,0.60,1.62,840.0,2
177,14.13,4.10,2.74,24.5,96.0,2.05,0.76,0.56,1.35,9.2,0.61,1.60,560.0,2


In [67]:
sample1 = test_df.iloc[0].tolist()
sample1 = sample1[:-1]
sample1

[13.17, 2.59, 2.37, 20.0, 120.0, 1.65, 0.68, 0.53, 1.46, 9.3, 0.6, 1.62, 840.0]

In [68]:
sample2 = test_df.iloc[1].tolist()
sample2 = sample2[:-1]
sample2

[14.13, 4.1, 2.74, 24.5, 96.0, 2.05, 0.76, 0.56, 1.35, 9.2, 0.61, 1.6, 560.0]

# Giving these 2 samples to the model

In [69]:
%%bash
data='[[13.27,4.28,2.26,20.0,120.0,1.59,0.69,0.43,1.35,10.2,0.59,1.56,835.0],[13.17,2.59,2.37,20.0,120.0,1.65,0.68,0.53,1.46,9.3,0.6,1.62,840.0]]'
echo $data

curl -d "{\"inputs\": $data}" -H 'Content-Type: application/json' 127.0.0.1:1594/invocations

[[13.27,4.28,2.26,20.0,120.0,1.59,0.69,0.43,1.35,10.2,0.59,1.56,835.0],[13.17,2.59,2.37,20.0,120.0,1.65,0.68,0.53,1.46,9.3,0.6,1.62,840.0]]


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   174  100    23  100   151  19557   125k --:--:-- --:--:-- --:--:--  169k


{"predictions": [2, 2]}

In [70]:
%%bash
data='[[13.27,4.28,2.26,20.0,120.0,1.59,0.69,0.43,1.35,10.2,0.59,1.56,835.0],[13.17,2.59,2.37,20.0,120.0,1.65,0.68,0.53,1.46,9.3,0.6,1.62,840.0]]'
echo $data

curl -d "{\"instances\": $data}" -H 'Content-Type: application/json' 127.0.0.1:1594/invocations

[[13.27,4.28,2.26,20.0,120.0,1.59,0.69,0.43,1.35,10.2,0.59,1.56,835.0],[13.17,2.59,2.37,20.0,120.0,1.65,0.68,0.53,1.46,9.3,0.6,1.62,840.0]]


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   177  100    23  100   154   9770  65420 --:--:-- --:--:-- --:--:-- 88500


{"predictions": [2, 2]}

# The model has made correct predictions!